In [1]:
import requests
from PIL import Image
import glob
import sys
sys.path.append("./lang-segment-anything")
from lang_sam import LangSAM

In [ ]:
img_list = glob.glob("../data/BGR/Profilbilder/*.JPG")
text_prompt = "ruler"
save_cropped_imgs = True
model = LangSAM(sam_type="vit_b") # b = base < l = large < h = huge

In [15]:
for img_file in img_list:

    try:
        image_pil = Image.open(img_file).convert("RGB")
        masks, boxes, phrases, logits = model.predict(image_pil, text_prompt)

        if len(masks) == 0:
            print(f"No '{text_prompt}' detected in the image.")
        else:
            x_min, y_min, x_max, y_max = boxes[0]
            width, height = image_pil.size

            if x_min > width // 2: # the ruler is on the right margin
                crop_area = (0, 0, int(x_min), height) # left, upper, right, lower
            else: # the ruler is on the left margin
                crop_area = (int(x_max), 0, width, height)

            cropped_img = image_pil.crop(crop_area)

            if save_cropped_imgs:
                cropped_img_file = img_file.replace('Profilbilder', 'Profilbilder_no_ruler')
                cropped_img.save(cropped_img_file)

    except (requests.exceptions.RequestException, IOError) as e:
        print(f"Error: {e}")

No 'ruler' detected in the image.
No 'ruler' detected in the image.
No 'ruler' detected in the image.
No 'ruler' detected in the image.
No 'ruler' detected in the image.


**Note 1**: 5 images did not have a real ruler on them, but an artificial marking stripe. These images were cropped manually.

In [22]:
img_list_no_ruler = glob.glob("../data/BGR/Profilbilder_no_ruler/*.JPG")
set_img = set( img.split('/')[-1] for img in img_list )
set_img_no_ruler = set( img.split('/')[-1] for img in img_list_no_ruler )
set_img - set_img_no_ruler

{'BY_3771_Y_1.JPG',
 'BY_3876_Y_1.JPG',
 'ST_1241_Y_1.JPG',
 'ST_1375_Y_1.JPG',
 'ST_2050_Y_1.JPG'}

**Note 2**: A handful of soil samples had extra objects on them e.g. shovel or name tags. These were cropped manually.

**Note 3**: A handful of images were wrongly cropped by SAM i.e. they were cropped down to dimension (1, width) or (height, 1). These were corrected manually.